In [2]:
import numpy as np
import opendssdirect as dss
from utils.device.Inverter import Inverter
from utils.controller.AdaptiveInvController import AdaptiveInvController
from utils.controller.FixedInvController import FixedInvController
import matplotlib.pyplot as plt
from math import tan,acos
import copy
import pandas as pd
import time

%matplotlib inline

# import numpy as np
# from matplotlib.mlab import PCA
from sklearn.decomposition import PCA

import scipy.linalg as spla


#######################################################
#######################################################
###Global variable initialization and error checking###
#######################################################
#######################################################
Sbase=1
LoadScalingFactor = 3
GenerationScalingFactor = 5
SlackBusVoltage = 1.04
NoiseMultiplyer= 1
#Set simulation analysis period - the simulation is from StartTime to EndTime
StartTime = 40600
EndTime = StartTime + 640
EndTime += 1 # creating a list, last element does not count, so we increase EndTime by 1
#Set hack parameters
TimeStepOfHack = 160
PercentHacked = np.array([0,0,0,0,0,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0,0,0,0,0,0,0,0,0,0,0,0,0])

#Set initial VBP parameters for uncompromised inverters
VBP_normal=np.array([0.97, 1, 1, 1.03, 1.06])
VBP_attack=np.array([0.997, 1, 1, 1.003, 1.007])

#Set delays for each node
Delay_VBPCurveShift = (30+2*np.random.randn(31)).astype(int)
#Delay_VBPCurveShift = (10+2*np.random.randn(31)).astype(int)
lpf_meas_vector = (1+0.2*np.random.randn(31))
lpf_output_vector= (0.1+0.015*np.random.randn(31))

#Set observer voltage threshold
ThreshHold_vqvp = 0.06
adaptive_gain=800


power_factor=0.9
pf_converted=tan(acos(power_factor))
# Number_of_Inverters = 13 #even feeder is 34Bus, we only have 13 inverters


#Error checking of the global variable -- TODO: add error handling here!
if EndTime < StartTime or EndTime < 0 or StartTime < 0:
    print('Setup Simulation Times Inappropriately.')
if NoiseMultiplyer < 0:
    print('Setup Noise Multiplyer Correctly.')
    
dss.run_command('Redirect feeder/IEEE_37_feeder_UB/ieee37.dss')

dss.Solution.Solve()
if not dss.Solution.Converged:
    print('Initial Solution Not Converged. Check Model for Convergence')
else:
    print('Initial Model Converged. Proceeding to Next Step.')
    #Doing this solve command is required for GridPV, that is why the monitors
    #go under a reset process
    dss.Monitors.ResetAll()
    
    #set solution Params
    #setSolutionParams(dss,'daily',1,1,'off',1000000,30000)
    dss.Solution.Mode(1)
    dss.Solution.Number(1)
    dss.Solution.StepSize(1)
    dss.Solution.ControlMode(-1)
    dss.Solution.MaxControlIterations(1000000)
    dss.Solution.MaxIterations(30000)
    #Easy process to get all names and count of loads, a trick to avoid
    #some more lines of code
    TotalLoads=dss.Loads.Count()
    AllLoadNames=dss.Loads.AllNames()
    print('OpenDSS Model Compliation Done.')

print('')
print(TotalLoads)
# dir(dss.Loads)

# (dss.Loads.AllNames())

# [x for x in dss.Circuit.AllBusNames() if 'bus' in x ]

# dss.Circuit.AllBusNames()

# dir(dss.Loads.AllNames)
print(len(dss.Loads.AllNames()))
print(dss.Loads.AllNames())

print(len(dss.Circuit.AllBusNames()))
print(dss.Circuit.AllBusNames())

print(dss.Bus.kVBase())

for k1 in range(len(dss.Circuit.AllBusNames())):
    if dss.Circuit.AllBusNames()[k1] == '702' or dss.Circuit.AllBusNames()[k1] == '720':
        dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k1])
#         print(dss.Bus.kVBase())
        tempVoltage = dss.Bus.VMagAngle()
        tempVoltage = tempVoltage
    #     print(tempVoltage)
#         print(tempVoltage[::2])
#         print(tempVoltage[0]*np.sqrt(3)/1)
        
        print(dss.Circuit.AllBusNames()[k1])
        print('BASE')
        print(dss.Bus.kVBase())
        print('VMAGANGLE')
        print(dss.Bus.VMagAngle())
        print('VOTLAGES')
        print(dss.Bus.Voltages())
        print('VLL')
        print(dss.Bus.puVLL())
        
        tempvoltage = dss.Bus.Voltages()
        
        Vab = tempvoltage[0]

    
# dir(dss.Bus)

Initial Model Converged. Proceeding to Next Step.
OpenDSS Model Compliation Done.

30
30
['s701ab', 's701bc', 's701ca', 's712ca', 's713ca', 's714ab', 's714bc', 's718ab', 's720ca', 's722bc', 's722ca', 's724bc', 's725bc', 's727ca', 's728abc', 's729ab', 's730ca', 's731bc', 's732ca', 's733ab', 's734ca', 's735ca', 's736bc', 's737ab', 's738ab', 's740ca', 's741ca', 's742ab', 's742bc', 's744ab']
39
['sourcebus', '799', '709', '775', '701', '702', '705', '713', '703', '727', '730', '704', '714', '720', '742', '712', '706', '725', '707', '724', '722', '708', '733', '732', '731', '710', '735', '736', '711', '741', '740', '718', '744', '734', '737', '738', '728', '729', '799r']
132.79056191361394
702
BASE
2.7712812921102037
VMAGANGLE
[2770.8271471070575, -0.016576386226687098, 2772.2106806315437, -120.00006412717475, 2770.8286481985665, 120.01646940345891]
VOTLAGES
[2770.8270311455412, -0.8016349670062316, -1386.1080276385633, -2400.803322540924, -1386.1040232048376, 2399.209668731197]
VLL
[0.8660

In [3]:
4.8/np.sqrt(3)

2.7712812921102037

In [36]:
A = np.random.randn(2,2)
A = np.array([[0, 1],[-2, -1]])

x = np.random.randn(2,8)
y = A@x

print(A)
print(x)
print(y)

print(y.T)
print(y.T.reshape((-1, 1), order="F"))

yls = y.T.reshape((-1, 1), order="F")
xls = spla.block_diag(x.T,x.T)

print(xls)

Bvec = spla.inv(xls.T@xls)@(xls.T@yls)

print(Bvec)

B = Bvec.reshape((2,2), order='F')
print(B)
A = B.T
print(A)

[[ 0  1]
 [-2 -1]]
[[ 2.29616045  0.52617245  0.6209244  -0.87879689  1.28524174 -0.39255846
  -0.04204305  0.26519546]
 [ 1.5015181  -0.3696468  -0.63198743  0.36628798  1.37670829 -1.41918426
   0.03618039 -1.28324629]]
[[ 1.5015181  -0.3696468  -0.63198743  0.36628798  1.37670829 -1.41918426
   0.03618039 -1.28324629]
 [-6.09383901 -0.6826981  -0.60986138  1.39130581 -3.94719177  2.20430118
   0.04790571  0.75285537]]
[[ 1.5015181  -6.09383901]
 [-0.3696468  -0.6826981 ]
 [-0.63198743 -0.60986138]
 [ 0.36628798  1.39130581]
 [ 1.37670829 -3.94719177]
 [-1.41918426  2.20430118]
 [ 0.03618039  0.04790571]
 [-1.28324629  0.75285537]]
[[ 1.5015181 ]
 [-0.3696468 ]
 [-0.63198743]
 [ 0.36628798]
 [ 1.37670829]
 [-1.41918426]
 [ 0.03618039]
 [-1.28324629]
 [-6.09383901]
 [-0.6826981 ]
 [-0.60986138]
 [ 1.39130581]
 [-3.94719177]
 [ 2.20430118]
 [ 0.04790571]
 [ 0.75285537]]
[[ 2.29616045  1.5015181   0.          0.        ]
 [ 0.52617245 -0.3696468   0.          0.        ]
 [ 0.6209244  -

In [56]:

Ts = 0.1

m = 1
b = 0.2
k = 10

Ac = np.array([[0, 1],[-k/m, -b/m]])

Ad = spla.expm(Ac*Ts)
Ad = np.eye(2) + Ts*Ac

print(Ad)

spla.eig(Ac)

time = Ts*np.arange(0,1001)

x = np.zeros((2,len(time)))

x[:,0] = [0.2, 0]

# print(len(time))
# print(x.shape)

for kt in range(len(time)-1):
    
    x[:,[kt+1]] = Ad@x[:,[kt]]
    
fig = plt.figure(figsize=[12,6])
plt.subplot(2,1,1)
plt.plot(time,x[0,:])
# plt.title('Node Active Power Injection')
# # plt.xlabel('Time')
# plt.ylabel('Active Power Injection [kW]')
plt.subplot(2,1,2)
plt.plot(time,x[1,:])
# plt.plot(time/3600,QinjTS[plotStartTimeStep:plotEndTimeStep,:])
# plt.title('Node Reactive Power Injection')
plt.xlabel('Time')
# plt.ylabel('Reactive Power Injection [kVAr]')
plt.show()


Ns = 2

for kt in range(0,5):
    
    xx = x[:,kt:kt+Ns]
#     print(xx)
    
    yy = x[:,kt+1:kt+1+Ns]
#     print(yy)

    xxblkdiag = spla.block_diag(xx.T,xx.T)
#     for k1 in range(0,1):
#         xxblkdiag = spla.block_diag(xxblkdiag,xx.T)
    print(xxblkdiag.shape)

    yyvec = yy.T.reshape((-1,1), order='F')
    print(yyvec.shape)

    Bvec = spla.inv(xxblkdiag.T@xxblkdiag)@(xxblkdiag.T@yyvec)
    
    A = Bvec.reshape((2,2), order='F').T
    print(A)
    
    
    
    
#     A = np.random.randn(2,2)
# A = np.array([[0, 1],[-2, -1]])

# x = np.random.randn(2,8)
# y = A@x

# print(A)
# print(x)
# print(y)

# print(y.T)
# print(y.T.reshape((-1, 1), order="F"))

# yls = y.T.reshape((-1, 1), order="F")
# xls = spla.block_diag(x.T,x.T)

# print(xls)

# Bvec = spla.inv(xls.T@xls)@(xls.T@yls)

# print(Bvec)

# B = Bvec.reshape((2,2), order='F')
# print(B)
# A = B.T
# print(A)
    

NameError: name 'T' is not defined